# Introduction
## Background
According to the National Cancer Institute, a stroke occurs when brain tissue is damaged due to a loss of blood flow to parts of the brain [1]. Worldwide, strokes are responsible for an annual mortality rate of 5.5 million. This makes it the 2nd leading cause of death. After the age of 55, the risk of a stroke doubles every decade, making age the strongest determinant. The leading risk factor of stroke is hypertension in both developing and developed nations [2].

## Question
Can we accurately predict the likelihood of a stroke in a patient given a set of clinical features ?

## Dataset description
The dataset that will be used is the Stroke Prediction Dataset, available via [Kaggle](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset), it consists of 5110 observation with 12 different variables including gender, age, hypertension, heart disease, marriage status, type of work (self-employed/private/children/government/never worked ), residence (rural or urban), average glucose levels, BMI, smoking status, and whether the patient has ever had a stroke.

# Preliminary exploratory data analysis

# Methodology
We will be performing a classification analysis to predict if a patient will have a stroke using these predictors:
| Variable               | Description                                                                  |
|------------------------|------------------------------------------------------------------------------|
| Gender                 | Whether the patient is a Male or Female                                      |
| Age                    | Age of the patient                                                           |
| Hypertension           | Whether the patient has been diagnosed with hypertension                     |
| Heart disease          | Whether the patient has been diagnosed with heart disease                    |
| Residence              | Whether the patient lives in a rural or urban area                           |
| Average glucose levels | Average glucose level in blood [mg/dL]                                       |
| BMI                    | Body mass index                                                              |
| Smoking status         | Whether the patient is a former smoker, has never smoked or currently smokes |
| Stroke                 | Whether the patient has had a stroke                                         |

We have opted to not use the following variables/column in our analysis:
- `ever_married`: This boolean column describes whether the individual has ever been married. We do not feel as though this is a strong indicator of stroke and/or general health outcomes.
- `work_type`: This column held the following possible values: `{Private, Self-employed, Govt_job}`. We did not feel as though the level at which this data was collected would contribute to predicting stroke - perhaps if this was separated between jobs that require manual labor (“blue-collar” jobs) vs. desk jobs (“white-collar” jobs) it could have been more useful.

## Possible visualizations
- **Bar charts** can be used to visualize the distribution of gender, hypertension, heart disease and other among stroke and non-stroke groups
- **Scatter plots** can be used to visualize relationships between age, average glucose levels, or BMI, with stroke occurrence as the color of the point
- **Heatmaps** can be used in combination with scatter plots to better visualize the different weights points can have within our predictive model

# Expected outcomes and significance
## What do we expect to find
There is quite a bit of research being done on the risk factors of stroke; however, there is not a consensus on this. For example, (this study)[https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2734855/] states that “smoking amplifies the positive association between BMI and CHD but not stroke” [3] conversely there are others studies who indicate that smoking “demonstrate[s] a strong association between smoking and stroke risk, with current smokers having at least a two- to fourfold increased risk of stroke”[4]. On the other hand, the notion of BMI being a good predictor of health is falling out of favor as of late. This is due to a conflict in findings which tell us that all cause mortality and BMI are strongly related [5], but higher levels of lean mass, which BMI fails to distinguish with fat mass [6], is inversely associated with mortality [7, 8]. We hope to discover a relationship between these factors (amongst others) on the risk of stroke in this dataset.

## Impacts of our findings
- **Early Intervention**: By being able to predict stroke risks, healthcare providers can better identify individuals who are at a higher risk of stroke before the deadly event happens. Early identification has the potential to significantly reduce the occurrence of strokes and further complication by allowing individuals to develop personalized treatment plans with medical professionals to improve their quality of life to minimize the chance.
- **Education + Awareness**: Our findings could be used to educate future generations of the risk factors that can lead to having a stroke. This can encourage them to live a healthier and positive lifestyle. Provides individuals, who may have risk factors that are genetic and not lifestyle, a better understanding of their risks. This will also benefit policy making, for example if the model identifies smoking as a strong predictor of a stroke, people in public health can prioritize initiatives that better control smoking.

## What future questions could this lead to
One key area for future exploration is the generalizability of the model in real-word scenarios. This is specifically important in the case if insurance companies/governments decide to use this technology as if the model isn’t generalizable it may lead to unfair outcomes and discriminate or have biases based on the original sample demographic it was trained with. As refined and well-trained the model can be, it's crucial to consider the potential exception and variation that arise due to the oddities of human biology and environmental factors. Therefore this research can lead to future research focusing on assessing the models ability across different populations in the world and healthcare systems.

# References
- [1] https://www.cancer.gov/publications/dictionaries/cancer-terms/def/stroke
- [2] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6288566/
- [3] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2734855/
-[4] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2928253/
[5] https://www.thelancet.com/journals/landia/article/PIIS2213-8587(18)30288-2/fulltext
[6] https://www.sciencedirect.com/science/article/pii/S2405525521000017
[7] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6348595/
[8] https://www.medicalnewstoday.com/articles/265215

# Loading Data and Dependencies

In [1]:
library(tidyverse)
library(tidymodels)
library(repr)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
url <- "https://raw.githubusercontent.com/21chanas3/dsci100-group73/c9c58b88eca13be20286f8bf70dce4668b878247/data/healthcare-dataset-stroke-data.csv"
if (!dir.exists("data")) {
    dir.create("data")
}
download.file(url, "data/stroke.csv")

In [3]:
stroke <- read_csv("data/stroke.csv")

Rows: 5110 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): gender, ever_married, work_type, Residence_type, bmi, smoking_status
dbl (6): id, age, hypertension, heart_disease, avg_glucose_level, stroke

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Cleaning and Wrangling

In [9]:
#Rename columns to all lowercase
stroke_colnames <- stroke |> colnames() |> make.names() |> tolower()
colnames(stroke) <- stroke_colnames

In [11]:
#Convert things to factors
stroke <- stroke |> mutate(gender = as_factor(gender), 
                 work_type = as_factor(work_type), 
                 residence_type = as_factor(residence_type), 
                 smoking_status = as_factor(smoking_status))

In [6]:
#For some reason bmi is of type char probably because of N/A, converting to double here
stroke <- stroke |> mutate(bmi = as.numeric(as.character(bmi)))

Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”


In [7]:
#Creating test/train split here
stroke_split <- initial_split(stroke, prop = 0.8, strata = stroke)
stroke_training <- training(stroke_split)
stroking_testing <- testing(stroke_split)